In [1]:
from hey_ac import HeyAC
import yaml

In [2]:
h = HeyAC(grammar_path = '../grammars/grammar_rom.txt')

In [3]:
texts = [
    'the speed of the fan is too low',
    'fan speed is low',
    'temperature is high',
    'humidity feels too high',
    'The room is very damp',
    'It is burning',
    'it is so cold',
    'the fan is not quite fast',
    'i feel very hot',
    'i am freezing',
    'the dog feels cold',
    'the ac is very weak',
    'the volume of air is so low',
]

In [4]:
for text in texts:
    print("="*50)
    print(text)
    print()
    print(yaml.dump(h.prune(text)))

the speed of the fan is too low

JJ: LOW
NEG: null
NN_OBJ: FAN
NN_PROP: VOLUME
VB: null

fan speed is low

JJ: LOW
NEG: null
NN_OBJ: FAN
NN_PROP: VOLUME
VB: null

temperature is high

JJ: HIGH
NEG: null
NN_OBJ: null
NN_PROP: TEMPERATURE
VB: null

humidity feels too high

JJ: HIGH
NEG: null
NN_OBJ: null
NN_PROP: HUMIDITY
VB: null

The room is very damp

JJ: HUMID
NEG: null
NN_OBJ: ENVIRONMENT
NN_PROP: null
VB: null

It is burning

JJ: HOT
NEG: null
NN_OBJ: ENVIRONMENT
NN_PROP: null
VB: null

it is so cold

JJ: COLD
NEG: null
NN_OBJ: ENVIRONMENT
NN_PROP: null
VB: null

the fan is not quite fast

JJ: FAST
NEG: NOT
NN_OBJ: FAN
NN_PROP: null
VB: null

i feel very hot

JJ: HOT
NEG: null
NN_OBJ: null
NN_PROP: null
VB: null

i am freezing

JJ: COLD
NEG: null
NN_OBJ: null
NN_PROP: null
VB: null

the dog feels cold

JJ: COLD
NEG: null
NN_OBJ: null
NN_PROP: null
VB: null

the ac is very weak

JJ: WEAK
NEG: null
NN_OBJ: AC
NN_PROP: null
VB: null

the volume of air is so low

JJ: LOW
NEG: null
NN_O

In [5]:
for text in texts:
    print("="*50)
    print(text)
    print(h.parse(text)[0])

the speed of the fan is too low
(S
  (NP
    (PROPERTY (DT the) (NN_PROP (VOLUME speed)))
    (IN of)
    (OBJECT (DT the) (NN_OBJ (FAN fan))))
  (VP (VBZ (BE is)) (ADJP (RB too) (JJ (LOW low)))))
fan speed is low
(S
  (NP
    (OBJECT (NN_OBJ (FAN fan)))
    (PROPERTY (NN_PROP (VOLUME speed))))
  (VP (VBZ (BE is)) (ADJP (JJ (LOW low)))))
temperature is high
(S
  (NP (PROPERTY (NN_PROP (TEMPERATURE temperature))))
  (VP (VBZ (BE is)) (ADJP (JJ (HIGH high)))))
humidity feels too high
(S
  (NP (PROPERTY (NN_PROP (HUMIDITY humidity))))
  (VP (VBZ (FEEL feels)) (ADJP (RB too) (JJ (HIGH high)))))
The room is very damp
(S
  (NP (OBJECT (DT the) (NN_OBJ (ENVIRONMENT room))))
  (VP (VBZ (BE is)) (ADJP (RB very) (JJ (HUMID damp)))))
It is burning
(S
  (NP (OBJECT (NN_OBJ (ENVIRONMENT it))))
  (VP (VBZ (BE is)) (ADJP (JJ (HOT burning)))))
it is so cold
(S
  (NP (OBJECT (NN_OBJ (ENVIRONMENT it))))
  (VP (VBZ (BE is)) (ADJP (RB so) (JJ (COLD cold)))))
the fan is not quite fast
(S
  (NP (OBJECT (DT 

In [6]:
parse_tree = h.parse(text)

In [7]:
for t in parse_tree[0]:
    print(t)

(NP
  (PROPERTY (DT the) (NN_PROP (VOLUME volume)))
  (IN of)
  (OBJECT (NN_OBJ (ENVIRONMENT air))))
(VP (VBZ (BE is)) (ADJP (RB so) (JJ (LOW low))))


In [8]:
list(parse_tree[0])

[Tree('NP', [Tree('PROPERTY', [Tree('DT', ['the']), Tree('NN_PROP', [Tree('VOLUME', ['volume'])])]), Tree('IN', ['of']), Tree('OBJECT', [Tree('NN_OBJ', [Tree('ENVIRONMENT', ['air'])])])]),
 Tree('VP', [Tree('VBZ', [Tree('BE', ['is'])]), Tree('ADJP', [Tree('RB', ['so']), Tree('JJ', [Tree('LOW', ['low'])])])])]